In [6]:
from kubeflow import fairing
from kubeflow.fairing.kubernetes import utils as k8s_utils
import uuid

fairing.config.set_preprocessor(
    'notebook', 
    notebook_file = 'mnist-dist.ipynb',
)

PRIVATE_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'
fairing.config.set_builder(
    'append',
    image_name='tfjob-fairing-mnist', 
    base_image=f'{PRIVATE_REGISTRY}/mybase',
    registry = PRIVATE_REGISTRY,
    push=True,
)

fairing.config.set_deployer(
    'tfjob',
    # namespace='mysapce', # 생략 시 현재 노트북의 네임스페이스 사용
    job_name=f'my-mnist-{uuid.uuid4().hex[:8]}',
    chief_count=1, 
    worker_count=2,
    pod_spec_mutators=[
        k8s_utils.mounting_pvc(pvc_name='mnist-tfjob-data-volume', pvc_mount_path='/data'),
        k8s_utils.get_resource_mutator(cpu=1, memory=5),
    ]
)

fairing.config.run()

[W 210224 07:15:46 utils:51] The function mounting_pvc has been deprecated,                     please use `volume_mounts`
[I 210224 07:15:46 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7f86f3e68cf8>
[I 210224 07:15:46 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f86f3e02550>
[I 210224 07:15:46 config:138] Using deployer: <kubeflow.fairing.deployers.tfjob.tfjob.TfJob object at 0x7f86f3de6fd0>
[W 210224 07:15:46 append:50] Building image using Append builder...
[I 210224 07:15:46 base:107] Creating docker context: /tmp/fairing_context_dt7tj1yd
[I 210224 07:15:46 converted_notebook:127] Converting mnist-dist.ipynb to mnist-dist.py
[I 210224 07:15:46 docker_creds_:234] Loading Docker credentials for repository 'sds.redii.net/mlopsdev/tmp:mnist-tf-0.2'
[W 210224 07:15:47 append:54] Image successfully built in 0.2387779038399458s.
[W 210224 07:15:47 append:94] Pus

2021-02-24 07:15:49.248917: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2021-02-24 07:15:49.249000: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2021-02-24 07:15:49.249008: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2021-02-24 07:15:50.072913: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-02-24 07:15:50.072937: E tensorflow/stream_executor/cuda

(<kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor at 0x7f86f3e68cf8>,
 <kubeflow.fairing.deployers.tfjob.tfjob.TfJob at 0x7f86f3de6fd0>)